### **🧠 Understanding Function Calling with OpenAI API (Function Tool Calling)**

In this lesson, we explore **Function Calling** with OpenAI's API — a powerful feature that allows the model to call specific functions based on user intent. This makes the model useful for building **AI agents**, **chatbots**, and **smart automation systems**.

### **🔧 What is Function Calling?**

Function calling lets the AI decide **when** and **how** to call an external function you've defined — based on the user's query. You define the function's name, parameters, and descriptions, and the model takes care of the rest.

### **🌦️ Example 1: `get_weather` Function**

This example shows how to define a simple function that gets the weather for a given location.

#### **🔍 Code:**
```python
from openai import OpenAI

client = OpenAI()

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": ["location"],
        }
    }
}]

# This is a conceptual example. The actual execution would involve another
# step to process the function call and return the result to the model.
# response = client.chat.completions.create(
#     model="gpt-4",
#     messages=[{"role": "user", "content": "What is the weather like in Paris today?"}],
#     tools=tools
# )

# print(response.choices[0].message)

```

#### **🧠 Explanation:**

* You define a tool named `get_weather` with one required parameter: `location`.
* The user asks about the weather in Paris.
* The model recognizes this intent and responds by indicating a call to the function with `{"location": "Paris"}`.
* The result would then be passed back to the user from a real API or a mock function.

### **📧 Example 2: `send_email` Function**

Now, let’s see how we define a tool for sending an email.

#### **🔍 Code:**
```python
from openai import OpenAI

client = OpenAI()

tools = [{
    "type": "function",
    "function": {
        "name": "send_email",
        "description": "Send an email to a recipient with a subject and body.",
        "parameters": {
            "type": "object",
            "properties": {
                "recipient": {
                    "type": "string",
                    "description": "The recipient's email address."
                },
                "subject": {
                    "type": "string",
                    "description": "The subject of the email."
                },
                "body": {
                    "type": "string",
                    "description": "The body content of the email."
                }
            },
            "required": ["recipient", "subject", "body"],
        }
    }
}]

# Conceptual example
# response = client.chat.completions.create(
#     model="gpt-4",
#     messages=[{"role": "user", "content": "Send an email to john@example.com about the meeting."}],
#     tools=tools
# )
#
# print(response.choices[0].message)
```

#### **🧠 Explanation:**

* The `send_email` tool requires 3 fields: `recipient`, `subject`, and `body`.
* The model will extract this information from the user's request and call the function accordingly.
* This could be used to automate real-world messaging systems with natural language.

### **Setup and Imports**

First, let's install the necessary libraries and set up our environment.

In [1]:
!pip install openai python-dotenv

In [2]:
import os
import json
import random
from openai import OpenAI
from dotenv import load_dotenv
import datetime
import pytz

# Load environment variables
load_dotenv()

# Initialize the OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def execute_function_call(user_query, tools):
    """A helper function to execute the full function calling flow."""
    messages = [{"role": "user", "content": user_query}]
    
    # First API call to the model
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    messages.append(response_message)
    
    # Check if the model wants to call a function
    if response_message.tool_calls:
        tool_call = response_message.tool_calls[0]
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        # Call the corresponding function
        if function_name == "convert_currency":
            function_response = convert_currency(**function_args)
        elif function_name == "get_current_time":
            function_response = get_current_time(**function_args)
        elif function_name == "get_stock_price":
            function_response = get_stock_price(**function_args)
        elif function_name == "evaluate_math_expression":
            function_response = evaluate_math_expression(**function_args)
        elif function_name == "get_random_joke":
            function_response = get_random_joke()
        elif function_name == "translate_text":
            function_response = translate_text(**function_args)
        elif function_name == "get_capital_city":
            function_response = get_capital_city(**function_args)
        elif function_name == "add_two_numbers":
            function_response = add_two_numbers(**function_args)
        elif function_name == "get_latest_news":
            function_response = get_latest_news(**function_args)
        elif function_name == "get_user_email":
            function_response = get_user_email(**function_args)
        else:
            function_response = f"Unknown function: {function_name}"
            
        # Send the function's result back to the model
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": function_name,
            "content": function_response
        })
        
        # Second API call to get a natural language response
        second_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages
        )
        return second_response.choices[0].message.content
    else:
        return response_message.content

### **Practice Questions**

Here are the solutions to the practice questions, demonstrating the complete function-calling flow for each.

#### **Question 1: Currency Conversion**

In [14]:
# Mock function for currency conversion
def convert_currency(amount, from_currency, to_currency):
    """Converts an amount from one currency to another."""
    rates = {"USD_to_EUR": 0.92, "EUR_to_USD": 1.09}
    key = f"{from_currency}_to_{to_currency}"
    if key in rates:
        converted_amount = amount * rates[key]
        return f"{amount} {from_currency} is equal to {converted_amount:.2f} {to_currency}."
    else:
        return f"Conversion from {from_currency} to {to_currency} is not supported."

# Tool definition for currency conversion
currency_tool = {
    "type": "function",
    "function": {
        "name": "convert_currency",
        "description": "Convert an amount from one currency to another.",
        "parameters": {
            "type": "object",
            "properties": {
                "amount": {"type": "number"},
                "from_currency": {"type": "string"},
                "to_currency": {"type": "string"}
            },
            "required": ["amount", "from_currency", "to_currency"]
        }
    }
}

# Execute the flow
user_query = "Convert 100 USD to EUR."
final_response = execute_function_call(user_query, [currency_tool])
print(final_response)

100 USD is equal to 92.00 EUR. Please note that exchange rates can fluctuate, so it's always a good idea to check for the most current rates if you are looking to make a transaction.


In [18]:
currency_conversion_prompts = [
    "How much is 50 USD in EUR?",
    "Can you convert 1500 JPY to USD for me?",
    "What's the equivalent of 25 GBP in CAD?",
    "I want to exchange 200 EUR for USD.",
    "If I have 100 AUD, how much NZD would I get?",
    "Show me the conversion for 75 CHF to EUR.",
    "Please convert 1,000,000 KRW to USD.",
    "What is the value of 800 CNY in JPY?",
    "I need to know the exchange rate for 300 SEK to NOK.",
    "Convert 5000 INR to USD please.",
    "How many EUR can I get for 120 USD?",
    "Tell me the result of changing 400 CAD to AUD.",
    "If I give you 60 EUR, how much is that in GBP?",
    "What's 90 USD in EUR today?",
    "I'm traveling and need to convert 1000 THB to EUR."
]

# Example of how to use these prompts
for prompt in currency_conversion_prompts:
    response = execute_function_call(prompt, [currency_tool])
    print(f"User: {prompt}\nAI: {response}\n---")

User: How much is 50 USD in EUR?
AI: 50 USD is equal to 46.00 EUR.
---
User: Can you convert 1500 JPY to USD for me?
AI: I'm unable to convert JPY to USD directly at the moment. You might want to use a currency conversion service or check a financial news website for the most recent rates.
---
User: What's the equivalent of 25 GBP in CAD?
AI: I'm unable to provide the real-time conversion rate from GBP to CAD. For the latest and most accurate exchange rates, please check a reliable financial news source or a currency conversion website. Alternatively, you can also use a financial app for the current rate.
---
User: I want to exchange 200 EUR for USD.
AI: If you exchange 200 EUR, you will get 218.00 USD.
---
User: If I have 100 AUD, how much NZD would I get?
AI: Unfortunately, it seems that I'm currently unable to perform a direct conversion from Australian Dollars (AUD) to New Zealand Dollars (NZD). However, you can easily find the exchange rate by checking a reliable financial news we

#### **Question 2: Current Time in Timezone**

In [4]:
# Mock function for getting the current time
def get_current_time(timezone):
    """Gets the current time in a specific timezone."""
    try:
        tz = pytz.timezone(timezone)
        current_time = datetime.datetime.now(tz)
        return f"The current time in {timezone} is {current_time.strftime('%H:%M:%S')}."
    except pytz.UnknownTimeZoneError:
        return f"Unknown timezone: {timezone}"

# Tool definition for getting the current time
time_tool = {
    "type": "function",
    "function": {
        "name": "get_current_time",
        "description": "Get the current time in a specific timezone.",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to get the current time for, e.g., 'America/New_York' or 'Asia/Tokyo'."
                }
            },
            "required": ["timezone"]
        }
    }
}

# Execute the flow
user_query = "What time is it in Tokyo?"
final_response = execute_function_call(user_query, [time_tool])
print(final_response)

The current time in Tokyo is 21:01 (9:01 PM).


In [19]:
current_time_prompts = [
    "What's the time in 'America/Los_Angeles'?",
    "Can you tell me the current time in 'Asia/Tokyo'?",
    "What time is it right now in London?",
    "I need to know the time in 'Australia/Sydney'.",
    "Tell me the time for 'Europe/Paris'.",
    "Is it morning or night in 'America/New_York'?",
    "What is the current hour in 'Asia/Dubai'?",
    "Give me the time for Moscow.",
    "Time in 'Africa/Cairo', please.",
    "Could you fetch the current time for 'Asia/Kolkata'?",
    "I have a meeting with someone in 'America/Chicago', what time is it there?",
    "Let me know the time in Berlin.",
    "What is the local time in 'Pacific/Auckland'?",
    "Current time in Shanghai please.",
    "I wonder what time it is in 'America/Sao_Paulo'."
]

# Example of how to use these prompts
for prompt in current_time_prompts:
    response = execute_function_call(prompt, [time_tool])
    print(f"User: {prompt}\nAI: {response}\n---")

User: What's the time in 'America/Los_Angeles'?
AI: The current time in Los Angeles (Pacific Time) is 6:33 AM.
---
User: Can you tell me the current time in 'Asia/Tokyo'?
AI: The current time in Asia/Tokyo is 22:33:33.
---
User: What time is it right now in London?
AI: The current time in London is 14:33 (2:33 PM).
---
User: I need to know the time in 'Australia/Sydney'.
AI: The current time in 'Australia/Sydney' is 23:33 (11:33 PM).
---
User: Tell me the time for 'Europe/Paris'.
AI: The current time in Europe/Paris is 15:33.
---
User: Is it morning or night in 'America/New_York'?
AI: It is morning in the 'America/New_York' timezone, as the current time is 09:33 AM.
---
User: What is the current hour in 'Asia/Dubai'?
AI: The current hour in 'Asia/Dubai' is 17 (5 PM).
---
User: Give me the time for Moscow.
AI: The current time in Moscow is 16:33.
---
User: Time in 'Africa/Cairo', please.
AI: The current time in Africa/Cairo is 16:33 (4:33 PM).
---
User: Could you fetch the current time 

#### **Question 3: Stock Price Lookup**

In [4]:
# Mock function for stock price lookup
def get_stock_price(company_name):
    """Gets the current stock price for a company."""
    prices = {"Apple": "175.20 USD", "Google": "135.50 USD"}
    return prices.get(company_name, f"Stock price for {company_name} not found.")

# Tool definition for stock price lookup
stock_tool = {
    "type": "function",
    "function": {
        "name": "get_stock_price",
        "description": "Get the current stock price for a company.",
        "parameters": {
            "type": "object",
            "properties": {
                "company_name": {
                    "type": "string",
                    "description": "The name of the company, e.g., 'Apple' or 'Google'."
                }
            },
            "required": ["company_name"]
        }
    }
}

# Execute the flow
user_query = "What's the stock price of Apple?"
final_response = execute_function_call(user_query, [stock_tool])
print(final_response)

As of the latest available data, the stock price of Apple is $175.20 USD.


In [5]:
stock_price_prompts = [
    "What's the current stock price for Microsoft?",
    "Can you look up the stock price for Amazon?",
    "Tell me the price of Tesla stock (TSLA).",
    "How is Apple's stock (AAPL) doing?",
    "I need the stock price for Google.",
    "What is the market price of NVIDIA stock?",
    "Fetch the stock value for Meta Platforms.",
    "Can you get me the stock price of Netflix?",
    "What about the stock of Johnson & Johnson?",
    "I'd like to know the stock price of Toyota.",
    "How much is one share of Coca-Cola?",
    "Please provide the stock price for Ford.",
    "What's the trading price for Samsung Electronics?",
    "Look up the stock for Berkshire Hathaway.",
    "Tell me the latest stock price for Alibaba."
]

for prompt in stock_price_prompts:
    response = execute_function_call(prompt, [stock_tool])
    print(f"User: {prompt}\nAI: {response}\n---")

User: What's the current stock price for Microsoft?
AI: I'm unable to retrieve the current stock price for Microsoft at the moment. You might want to check a financial news website or a stock trading platform for the latest information.
---
User: Can you look up the stock price for Amazon?
AI: I'm unable to look up the current stock price for Amazon. To find the most recent stock price, please check a financial news website, your brokerage platform, or a stock market app.
---
User: Tell me the price of Tesla stock (TSLA).
AI: I'm unable to provide the current stock price for Tesla (TSLA) at this moment. For the most accurate and up-to-date information, I recommend checking a reliable financial news website, stock market app, or consulting with a financial advisor.
---
User: How is Apple's stock (AAPL) doing?
AI: As of the latest available information, Apple's stock (AAPL) is trading at 175.20 USD. Please note that stock prices are subject to change, so it would be advisable to check a 

#### **Question 4: Math Calculation**

In [7]:
# Mock function for math calculation
def evaluate_math_expression(expression):
    """Evaluates a math expression."""
    try:
        # Using eval is generally unsafe, but for this controlled example, it's fine.
        result = eval(expression)
        return f"The result of '{expression}' is {result}."
    except:
        return "Invalid math expression."

# Tool definition for math calculation
math_tool = {
    "type": "function",
    "function": {
        "name": "evaluate_math_expression",
        "description": "Evaluate a math expression.",
        "parameters": {
            "type": "object",
            "properties": {
                "expression": {
                    "type": "string",
                    "description": "The math expression to evaluate, e.g., '15 * 12'."
                }
            },
            "required": ["expression"]
        }
    }
}

# Execute the flow
user_query = "What is 15 * 12?"
final_response = execute_function_call(user_query, [math_tool])
print(final_response)

15 multiplied by 12 is 180.


In [8]:
ath_expression_prompts = [
    "Can you calculate '5 + 8 * 3'?",
    "What is the result of '(100 - 25) / 5'?",
    "Evaluate '2**10'.",
    "Solve this: '144**0.5'.",
    "What is '15 * (4 + 6)'?",
    "I need to compute '30 / 2 + 5'.",
    "Can you tell me what '7 * 8' is?",
    "Please calculate '500 / 25'.",
    "What is '99 + 1'?",
    "Evaluate the expression '120 - 45'.",
    "What's '18 / 3'?",
    "I need to know the value of '4 * 5 * 6'.",
    "Calculate '1000 / 10 * 2'.",
    "What's the answer to '25 - 5 * 2'?",
    "Solve '7 + 7 + 7'."
]

for prompt in ath_expression_prompts:
    response = execute_function_call(prompt, [math_tool])
    print(f"User: {prompt}\nAI: {response}\n---")

User: Can you calculate '5 + 8 * 3'?
AI: The result of the expression \(5 + 8 \times 3\) is 29.
---
User: What is the result of '(100 - 25) / 5'?
AI: The result of \((100 - 25) / 5\) is 15.0.
---
User: Evaluate '2**10'.
AI: The result of \(2^{10}\) is 1024.
---
User: Solve this: '144**0.5'.
AI: The result of \( 144^{0.5} \) is 12.0.
---
User: What is '15 * (4 + 6)'?
AI: The result of \(15 \times (4 + 6)\) is 150.
---
User: I need to compute '30 / 2 + 5'.
AI: The result of the expression \( \frac{30}{2} + 5 \) is 20.
---
User: Can you tell me what '7 * 8' is?
AI: The result of \(7 \times 8\) is 56.
---
User: Please calculate '500 / 25'.
AI: The result of \( 500 \div 25 \) is 20.
---
User: What is '99 + 1'?
AI: The result of \( 99 + 1 \) is 100.
---
User: Evaluate the expression '120 - 45'.
AI: The result of the expression \( 120 - 45 \) is 75.
---
User: What's '18 / 3'?
AI: The result of \( \frac{18}{3} \) is 6.
---
User: I need to know the value of '4 * 5 * 6'.
AI: The value of \(4 \ti

#### **Question 5: Get a Random Joke**

In [10]:
# Mock function for getting a random joke
def get_random_joke():
    """Fetches a random joke."""
    jokes = [
        "Why don't scientists trust atoms? Because they make up everything!",
        "Why did the scarecrow win an award? Because he was outstanding in his field!",
        "What do you call a lazy kangaroo? Pouch potato."
    ]
    return random.choice(jokes)

# Tool definition for getting a random joke
joke_tool = {
    "type": "function",
    "function": {
        "name": "get_random_joke",
        "description": "Get a random joke.",
        "parameters": {
            "type": "object",
            "properties": {}
        }
    }
}

# Execute the flow
user_query = "Tell me a joke."
final_response = execute_function_call(user_query, [joke_tool])
print(final_response)

What do you call a lazy kangaroo? Pouch potato.


In [11]:
random_joke_prompts = [
    "Tell me a funny joke.",
    "Can you make me laugh?",
    "I want to hear a joke.",
    "Hit me with your best joke.",
    "Do you know any good jokes?",
    "Tell me something funny.",
    "Joke of the day?",
    "I'm bored, tell me a joke.",
    "Let's have a joke.",
    "Can I get a joke, please?",
    "Make me chuckle.",
    "Got any jokes?",
    "I need a laugh.",
    "Share a random joke.",
    "Give me a one-liner."
]

for prompt in random_joke_prompts:
    response = execute_function_call(prompt, [joke_tool])
    print(f"User: {prompt}\nAI: {response}\n---")

User: Tell me a funny joke.
AI: Why did the scarecrow win an award? Because he was outstanding in his field!
---
User: Can you make me laugh?
AI: Why don't scientists trust atoms? Because they make up everything!
---
User: I want to hear a joke.
AI: Why did the scarecrow win an award? Because he was outstanding in his field!
---
User: Hit me with your best joke.
AI: What do you call a lazy kangaroo? Pouch potato.
---
User: Do you know any good jokes?
AI: Here's a joke for you: Why did the scarecrow win an award? Because he was outstanding in his field!
---
User: Tell me something funny.
AI: Here's a joke for you: Why don't scientists trust atoms? Because they make up everything!
---
User: Joke of the day?
AI: Why did the scarecrow win an award? Because he was outstanding in his field!
---
User: I'm bored, tell me a joke.
AI: Why did the scarecrow win an award? Because he was outstanding in his field!
---
User: Let's have a joke.
AI: What do you call a lazy kangaroo? Pouch potato.
---
U

#### **Question 6: Text Translation**

In [8]:
# Mock function for text translation
def translate_text(text, target_language):
    """Translates text to a target language."""
    translations = {"Hello, how are you?": "Hola, ¿cómo estás?"}
    return translations.get(text, "Translation not available.")

# Tool definition for text translation
translate_tool = {
    "type": "function",
    "function": {
        "name": "translate_text",
        "description": "Translate text from English to Spanish.",
        "parameters": {
            "type": "object",
            "properties": {
                "text": {"type": "string"},
                "target_language": {"type": "string"}
            },
            "required": ["text", "target_language"]
        }
    }
}

# Execute the flow
user_query = "Translate 'Hello, how are you?' to Spanish."
final_response = execute_function_call(user_query, [translate_tool])
print(final_response)

The translation of "Hello, how are you?" to Spanish is "Hola, ¿cómo estás?"


#### **Question 7: Capital of a Country**

In [13]:
# Mock function for getting the capital of a country
def get_capital_city(country):
    """Gets the capital city of a country."""
    capitals = {"Japan": "Tokyo", "France": "Paris"}
    return capitals.get(country, f"Capital of {country} not found.")

# Tool definition for getting the capital of a country
capital_tool = {
    "type": "function",
    "function": {
        "name": "get_capital_city",
        "description": "Get the capital city of a country.",
        "parameters": {
            "type": "object",
            "properties": {
                "country": {"type": "string"}
            },
            "required": ["country"]
        }
    }
}

# Execute the flow
user_query = "What is the capital of Japan?"
final_response = execute_function_call(user_query, [capital_tool])
print(final_response)

The capital of Japan is Tokyo.


In [14]:
apital_city_prompts = [
    "What is the capital of Canada?",
    "Tell me the capital city of Australia.",
    "What's the capital of Italy?",
    "I want to know the capital of South Korea.",
    "What is the main city of Germany?",
    "Capital of Russia?",
    "Can you tell me the capital of Brazil?",
    "What is the capital of the United Kingdom?",
    "I'm trying to remember the capital of Argentina.",
    "What city is the capital of Egypt?",
    "Tell me the capital of Thailand.",
    "What's the capital of Spain?",
    "I need to know the capital of Mexico.",
    "Capital city of India?",
    "What is the capital of Nigeria?"
]

for prompt in apital_city_prompts:
    response = execute_function_call(prompt, [capital_tool])
    print(f"User: {prompt}\nAI: {response}\n---")

User: What is the capital of Canada?
AI: The capital of Canada is Ottawa.
---
User: Tell me the capital city of Australia.
AI: The capital city of Australia is Canberra.
---
User: What's the capital of Italy?
AI: The capital of Italy is Rome.
---
User: I want to know the capital of South Korea.
AI: The capital of South Korea is Seoul.
---
User: What is the main city of Germany?
AI: The main city, or capital, of Germany is Berlin.
---
User: Capital of Russia?
AI: The capital of Russia is Moscow.
---
User: Can you tell me the capital of Brazil?
AI: The capital of Brazil is Brasília.
---
User: What is the capital of the United Kingdom?
AI: The capital of the United Kingdom is London.
---
User: I'm trying to remember the capital of Argentina.
AI: The capital of Argentina is Buenos Aires.
---
User: What city is the capital of Egypt?
AI: The capital city of Egypt is Cairo.
---
User: Tell me the capital of Thailand.
AI: The capital of Thailand is Bangkok.
---
User: What's the capital of Spain

#### **Question 8: Latest News Headline**

In [16]:
# Mock function for getting the latest news
def get_latest_news(topic):
    """Gets the latest news on a topic."""
    return f"Latest news on {topic}: AI models are becoming increasingly powerful."

# Tool definition for getting the latest news
news_tool = {
    "type": "function",
    "function": {
        "name": "get_latest_news",
        "description": "Get the latest news on a topic.",
        "parameters": {
            "type": "object",
            "properties": {
                "topic": {"type": "string"}
            },
            "required": ["topic"]
        }
    }
}

# Execute the flow
user_query = "What's the latest news on AI?"
final_response = execute_function_call(user_query, [news_tool])
print(final_response)

The latest news on AI is that AI models are becoming increasingly powerful. If you have a specific area of AI you're interested in, feel free to ask!


In [17]:
latest_news_prompts = [
    "What's the latest news in technology?",
    "Any breaking news on climate change?",
    "Tell me the top news stories about space exploration.",
    "I want to know the latest news in global politics.",
    "What's new in the world of healthcare?",
    "Give me the latest update on the stock market.",
    "News about artificial intelligence, please.",
    "What are the latest developments in renewable energy?",
    "I'm interested in news about the movie industry.",
    "What's the latest in sports news?",
    "Can you find recent news about scientific discoveries?",
    "Latest news headlines about business?",
    "What's happening in the world of fashion?",
    "Are there any important updates on public health?",
    "Tell me the latest on video games."
]

for prompt in latest_news_prompts:
    response = execute_function_call(prompt, [news_tool])
    print(f"User: {prompt}\nAI: {response}\n---")

User: What's the latest news in technology?
AI: The latest news in technology highlights that AI models are becoming increasingly powerful.
---
User: Any breaking news on climate change?
AI: I don't have the latest update on climate change itself at the moment, but there's news highlighting the role of AI models, which are growing increasingly powerful, possibly playing a significant role in climate change analysis and solutions. If you have more specific interests or areas you want to explore regarding climate change, let me know!
---
User: Tell me the top news stories about space exploration.
AI: I currently don't have real-time access to the news. However, I recommend checking major news sources, space agencies like NASA or ESA, or specialized space news sites like Space.com for the latest and most detailed updates on space exploration.
---
User: I want to know the latest news in global politics.
AI: The latest development in global politics involves the increasing power and influen

#### **Question 9: Get User Email**

In [19]:
# Mock function for getting a user's email
def get_user_email(username):
    """Retrieves a user's email by username."""
    emails = {"john_doe": "john.doe@example.com"}
    return emails.get(username, f"Email for {username} not found.")

# Tool definition for getting a user's email
email_tool = {
    "type": "function",
    "function": {
        "name": "get_user_email",
        "description": "Get a user's email by username.",
        "parameters": {
            "type": "object",
            "properties": {
                "username": {"type": "string"}
            },
            "required": ["username"]
        }
    }
}

# Execute the flow
user_query = "What's the email for user john_doe?"
final_response = execute_function_call(user_query, [email_tool])
print(final_response)

The email for user "john_doe" is john.doe@example.com.


In [20]:
get_email_prompts = [
    "What is the email address for the user 'sara_jones'?",
    "Can you find the email for 'admin_user'?",
    "I need the email of 'test_account_123'.",
    "Look up the email for username 'developer_x'.",
    "What's the email associated with 'jane_doe'?",
    "Find me the email of 'support_team'.",
    "What is 'michael_smith's email?",
    "I need the contact email for user 'finance_dept'.",
    "Can you get the email for the username 'guest001'?",
    "What is the email for 'project_manager_z'?",
    "Tell me the email of 'sales_lead'.",
    "Find email for user 'hr_recruiter'.",
    "What's the email for the user named 'alex'?",
    "I need to find the email for 'beta_tester_5'.",
    "Get me the email of 'david_chen'."
]

for prompt in get_email_prompts:
    response = execute_function_call(prompt, [email_tool])
    print(f"User: {prompt}\nAI: {response}\n---")

User: What is the email address for the user 'sara_jones'?
AI: I'm sorry, I couldn't retrieve the email address for the user 'sara_jones'.
---
User: Can you find the email for 'admin_user'?
AI: I'm sorry, but I can't find the email address for 'admin_user'. Is there anything else I might be able to help you with?
---
User: I need the email of 'test_account_123'.
AI: I'm sorry, but I couldn't find the email for 'test_account_123'. If you have any other questions or need further assistance, feel free to let me know!
---
User: Look up the email for username 'developer_x'.
AI: The email address for the username 'developer_x' could not be found. If you have any other information or need further assistance, feel free to ask!
---
User: What's the email associated with 'jane_doe'?
AI: I'm sorry, but I don't have access to the email associated with 'jane_doe'.
---
User: Find me the email of 'support_team'.
AI: I'm sorry, but I couldn't find an email address for "support_team." If this is an int